In [1]:
using Gurobi, JuMP, DataFrames, CSV, Plots, Clustering

Read in the collected data; demand, supply, and weighting are collected at the sources given in the #readme

In [2]:
demand = Matrix(CSV.read("data/gurobi/demand.csv", DataFrame))
supply = Matrix(CSV.read("data/gurobi/supplyT.csv", DataFrame))
weighting = Matrix(CSV.read("data/gurobi/weighting.csv", DataFrame))
inverse_populations = Matrix(CSV.read("data/gurobi/inverse_populations.csv", DataFrame));
loc = CSV.read("./data/locations.csv", DataFrame);

In [3]:
function distance(x1, y1, x2, y2)
    floatpointcorrection = sin(y1)*sin(y2) + cos(y1)*cos(y2)*cos(x1-x2)
    floatpointcorrection = ifelse(floatpointcorrection > 1, 1.0, floatpointcorrection)
    floatpointcorrection = ifelse(floatpointcorrection < -1, -1.0, floatpointcorrection)
    ß = acos(floatpointcorrection)
    return ß * 6378.1
end;

In [4]:
demand = vcat(demand,zeros(8,4))
supply = vcat(supply,zeros(8,74))
inverse_populations = vcat(inverse_populations,zeros(8,1))
dog = CSV.read("./data/cluster_centers.csv", DataFrame)
dog[!,:Area] = ["Hub1","Hub2","Hub3","Hub4","Hub5","Hub6","Hub7","Hub8"]
loc = vcat(loc, dog)

Row,Area,latitude,longitude
,String,Float64,Float64
1,afghanistan,33.94,67.71
2,albania,41.15,20.17
3,algeria,28.03,1.66
4,angola,-11.2,17.87
5,antigua and barbuda,17.06,-61.8
6,argentina,-38.42,-63.62
7,armenia,40.07,45.04
8,australia,-25.27,133.78
9,austria,47.52,14.55


In [5]:
arcDistance = DataFrame(zeros(176, 176), :auto)
for i in 1:176
    for j in 1:176
        arcDistance[i, j] = distance(loc[i,2], loc[i,3], loc[j,2], loc[j,3])
    end
end
arcDistance = Matrix(round.(arcDistance, digits=7));

In [9]:
using Tables

In [12]:
CSV.write("data/gurobi/arcDistance.csv", Tables.table(arcDistance))
CSV.write("data/gurobi/demand.csv", Tables.table(demand))
CSV.write("data/gurobi/supplyT.csv", Tables.table(supply))
CSV.write("data/gurobi/weighting.csv", Tables.table(weighting))
CSV.write("data/gurobi/inverse_populations.csv", Tables.table(inverse_populations));

In [ ]:
G = 10000000
H = 78.91
å = 0.075
øH= 0.6
lambda = 250 * 365
n = 176
p = 74
q = 8
K = 74
uncertainty = 50
inverse_foods = inv.([41.3, 107.38, 37.9961, 330.4])

In [27]:
function ∑(x)
    return sum(x)
end

∑ (generic function with 1 method)

In [30]:
model = Model(Gurobi.Optimizer)
# set_time_limit_sec(model, 60.0)
@variable(model, X[i=1:n,j=1:n,k=1:p] .≥ 0)
@variable(model, Y[i=1:n,j=1:n,k=1:p] .≥ 0)
@variable(model, Z[i=1:n,j=1:n,k=1:p] .≥ 0)
@variable(model, ß ≥ 0) # integer = true

@constraint(model, hubRestriction[l=1:168], ∑(∑(Y[:,l,:])) ≤ 0)
@constraint(model, hubRestriction1[l=1:168], ∑(∑(Z[l,:,:])) ≤ 0)
@constraint(model, hubEquality[l=169:n,k=1:p], ∑(Y[:,l,k]) .== ∑(Z[l,:,k]))
@constraint(model, realSupply, ∑(X[:,l,:] for l=1:168) + ∑(Y[:,l,:] for l=169:n) .≤ abs.(supply))

@constraint(model, auxiliary, ß .≥ inverse_populations * (demand - G * (∑((X + Z)[l,:,:] for l=1:n)) * weighting) * inverse_foods)

@objective(model, Min, å * H * ∑(
    ∑( arcDistance .* ∑(X[i,j,:]) for j=1:168 ) + 
    ∑( arcDistance .* ∑((Y.*øH)[i,j,:] ) for j=169:n ) + 
    ∑( arcDistance .* ∑(Z[j,i,:]) for j=169:n ) for i=1:168) + 
    (1-å) * ß * lambda)

optimize!(model)